In [1]:
%%html
<style>
.dataframe td {
    white-space: nowrap;
}
</style>

In [2]:
import pandas as pd
import numpy as np

from IPython.display         import display, Markdown, Latex

pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)

print('Done')

Done


In [3]:
files = [
    # Regressão simbólica com algoritmos genéticos
    # ITEA    - faltando
    # FEAT    - faltando
    # SymTree - faltando
    'GSGP-resultsregression(novo)',
    'gplearn-resultsregression',
    'dcgp-resultsregression',
    
    # Árvores/knn
    'forest-resultsregression',
    'knn-resultsregression',
    'tree-resultsregression',
    
    # Regressores lineares
    'elnet-resultsregression',
    'lasso-resultsregression',
    'lassolars-resultsregression',
    'ridge-resultsregression',
]

def highlight_min(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_min = s == s.min()
    return ['background-color: lightgreen' if v else '' for v in is_min]


In [18]:
RMSES_train = []
RMSES_test  = []

for file in files:
    print(file)
    fname = f'{file}.csv'

    # Arquivo com RMSE de treino e teste da melhor configuração para cada par treino-teste
    resultsDF = pd.read_csv(fname)
    
    # Vendo se é o caso de algoritmo estocástico (várias repetições sobre cada par treino-teste)
    # ou de determinísticos (única repetição para cada par)
    if 'Rep' in resultsDF.columns:
        print('Estocástico')
        resultsDF_ = resultsDF.set_index(['dataset', 'Fold', 'Rep'])

        # Tirando a média da melhor configuração em cada fold (e descartando 2 primeiras colunas, configuração e cv)
        resultsDF_median = resultsDF_.groupby(['dataset']).mean().iloc[:, 2:]
        resultsDF_median.columns = ['RMSE_train_mean', 'RMSE_test_mean']

        # Colocando o desvio padrão e tirando as 2 primeiras colunas (fold e rep, não interessam)
        resultsDF_std = resultsDF_.groupby(['dataset']).std().iloc[:, 2:]
        resultsDF_std.columns = ['RMSE_train_std', 'RMSE_test_std']

        # juntando tudo em um só
        resultsDF_ = pd.merge(resultsDF_median, resultsDF_std, left_index=True, right_index=True)
        
        print(resultsDF_)
    else:
        print('Determinístico')
        # Calculando as medianas e tirando a coluna de fold (configuração já some pois não é numérico)
        # Note que a conf não é numérica apenas em algoritmos rápidos (os lineares)
        resultsDF_median = resultsDF.groupby('dataset').mean().iloc[:, 1:]
        resultsDF_median.columns = ['RMSE_train_mean', 'RMSE_test_mean']

        # Colocando o desvio padrão 
        resultsDF_std = resultsDF.groupby('dataset').std().iloc[:, 1:]
        resultsDF_std.columns = ['RMSE_train_std', 'RMSE_test_std']

        # juntando tudo em um só
        resultsDF_ = pd.merge(resultsDF_median, resultsDF_std, left_index=True, right_index=True)
        print(resultsDF_)
    
    RMSES_train.append(pd.Series(
        data = [f'${str(round(mean, 2))}\pm{str(round(std, 2))}$' for mean, std in 
                zip(resultsDF_['RMSE_train_mean'].values, resultsDF_['RMSE_train_std'].values)],
        index = resultsDF_.index,
        name = file.replace('-resultsregression', '')
    ))
    
    RMSES_test.append(pd.Series(
        data = [f'${str(round(mean, 2))}\pm{str(round(std, 2))}$' for mean, std in 
                zip(resultsDF_['RMSE_test_mean'].values, resultsDF_['RMSE_test_std'].values)],
        index = resultsDF_.index,
        name = file.replace('-resultsregression', '')
    ))

GSGP-resultsregression(novo)
Estocástico
               RMSE_train_mean  RMSE_test_mean  RMSE_train_std  RMSE_test_std
dataset                                                                      
Geographical         42.032963       44.350340        0.960309       1.507486
airfoil              10.597684       10.597991        4.621630       4.350163
concrete              7.240097        7.857217        0.738366       1.097733
energyCooling         2.067702        2.130404        0.431474       0.420648
energyHeating         1.500970        1.571454        0.451373       0.457086
tecator               1.042092        1.410030        0.462202       0.562009
towerData            63.838370       63.997013        4.702326       5.151672
wineRed               0.624968        0.643030        0.006544       0.034485
wineWhite             0.733948        0.739867        0.015091       0.017716
yacht                 8.809734        9.032945        1.108111       1.034298
gplearn-resultsregressi

In [19]:
display(Markdown('# Partição de treino'))
pd.concat(RMSES_train, axis=1).T.style.apply(highlight_min)

# Partição de treino

dataset,Geographical,airfoil,concrete,energyCooling,energyHeating,tecator,towerData,wineRed,wineWhite,yacht
GSGP(novo),$42.03\pm0.96$,$10.6\pm4.62$,$7.24\pm0.74$,$2.07\pm0.43$,$1.5\pm0.45$,$1.04\pm0.46$,$63.84\pm4.7$,$0.62\pm0.01$,$0.73\pm0.02$,$8.81\pm1.11$
gplearn,$49.66\pm2.58$,$10.28\pm7.26$,$13.64\pm1.89$,$4.44\pm0.91$,$4.67\pm0.92$,$3.0\pm0.63$,$68.34\pm5.87$,$0.79\pm0.06$,$0.87\pm0.05$,$6.38\pm2.45$
dcgp,$55.6\pm3.41$,$7.56\pm1.33$,$19.45\pm2.42$,$12.89\pm2.35$,$13.74\pm2.02$,$4.1\pm0.35$,$106.34\pm24.03$,$1.14\pm0.97$,$1.14\pm0.84$,$17.09\pm1.76$
forest,$11.26\pm0.76$,$1.8\pm0.03$,$2.97\pm0.07$,$1.22\pm0.03$,$0.4\pm0.01$,$0.63\pm0.02$,$12.97\pm0.42$,$0.34\pm0.0$,$0.53\pm0.0$,$0.4\pm0.02$
knn,$31.51\pm0.52$,$5.9\pm0.08$,$8.27\pm0.09$,$1.97\pm0.04$,$2.37\pm0.06$,$1.39\pm0.04$,$15.11\pm0.14$,$0.68\pm0.01$,$0.71\pm0.01$,$8.75\pm0.68$
tree,$19.73\pm3.5$,$2.04\pm0.08$,$2.9\pm0.16$,$1.68\pm0.07$,$0.34\pm0.03$,$0.66\pm0.21$,$13.55\pm0.52$,$0.6\pm0.01$,$0.7\pm0.01$,$0.33\pm0.41$
elnet,$43.7\pm0.18$,$4.82\pm0.04$,$10.47\pm0.12$,$3.22\pm0.02$,$2.96\pm0.05$,$0.8\pm0.03$,$30.66\pm1.03$,$0.65\pm0.01$,$0.75\pm0.0$,$8.94\pm0.15$
lasso,$43.85\pm0.27$,$4.82\pm0.04$,$10.47\pm0.12$,$3.22\pm0.02$,$2.96\pm0.05$,$0.82\pm0.06$,$30.87\pm1.14$,$0.65\pm0.01$,$0.75\pm0.0$,$8.94\pm0.15$
lassolars,$44.42\pm0.11$,$4.82\pm0.04$,$10.47\pm0.12$,$3.22\pm0.02$,$2.96\pm0.05$,$0.8\pm0.06$,$30.65\pm1.23$,$0.65\pm0.01$,$0.75\pm0.0$,$8.95\pm0.16$
ridge,$43.72\pm0.26$,$4.82\pm0.04$,$10.47\pm0.12$,$3.22\pm0.02$,$2.96\pm0.05$,$0.8\pm0.04$,$30.31\pm1.05$,$0.65\pm0.01$,$0.75\pm0.0$,$8.95\pm0.16$


In [20]:
display(Markdown('# Partição de teste'))
pd.concat(RMSES_test, axis=1).T.style.apply(highlight_min)

# Partição de teste

dataset,Geographical,airfoil,concrete,energyCooling,energyHeating,tecator,towerData,wineRed,wineWhite,yacht
GSGP(novo),$44.35\pm1.51$,$10.6\pm4.35$,$7.86\pm1.1$,$2.13\pm0.42$,$1.57\pm0.46$,$1.41\pm0.56$,$64.0\pm5.15$,$0.64\pm0.03$,$0.74\pm0.02$,$9.03\pm1.03$
gplearn,$49.72\pm2.94$,$10.31\pm7.21$,$13.65\pm2.09$,$4.46\pm0.9$,$4.67\pm0.97$,$3.05\pm0.7$,$68.61\pm6.59$,$0.79\pm0.07$,$0.88\pm0.05$,$6.68\pm2.94$
dcgp,$63.75\pm46.9$,$7.55\pm1.38$,$19.5\pm2.27$,$12.84\pm2.25$,$13.67\pm1.88$,$4.1\pm0.43$,$106.93\pm24.39$,$1.14\pm0.97$,$1.14\pm0.83$,$17.01\pm2.2$
forest,$25.78\pm3.65$,$2.37\pm0.07$,$5.25\pm0.32$,$1.79\pm0.12$,$0.57\pm0.1$,$1.25\pm0.12$,$17.69\pm1.27$,$0.58\pm0.04$,$0.66\pm0.01$,$1.07\pm0.31$
knn,$31.17\pm0.87$,$5.86\pm0.26$,$7.85\pm0.73$,$1.92\pm0.13$,$2.32\pm0.42$,$1.29\pm0.09$,$14.81\pm0.85$,$0.66\pm0.05$,$0.69\pm0.02$,$7.56\pm1.31$
tree,$31.92\pm7.42$,$2.93\pm0.1$,$6.5\pm0.5$,$1.85\pm0.14$,$0.57\pm0.07$,$1.68\pm0.15$,$20.09\pm1.02$,$0.69\pm0.06$,$0.75\pm0.02$,$1.44\pm0.26$
elnet,$43.21\pm1.16$,$4.82\pm0.17$,$10.44\pm0.49$,$3.21\pm0.11$,$2.94\pm0.19$,$0.75\pm0.08$,$31.17\pm3.5$,$0.65\pm0.03$,$0.75\pm0.01$,$8.97\pm0.71$
lasso,$43.28\pm1.02$,$4.82\pm0.17$,$10.44\pm0.49$,$3.21\pm0.11$,$2.94\pm0.19$,$0.76\pm0.09$,$30.92\pm3.67$,$0.65\pm0.03$,$0.75\pm0.01$,$8.94\pm0.7$
lassolars,$43.92\pm1.01$,$4.82\pm0.17$,$10.44\pm0.49$,$3.21\pm0.11$,$2.94\pm0.19$,$0.74\pm0.08$,$30.56\pm3.46$,$0.65\pm0.03$,$0.75\pm0.01$,$8.93\pm0.71$
ridge,$42.95\pm1.26$,$4.82\pm0.17$,$10.44\pm0.49$,$3.21\pm0.11$,$2.94\pm0.19$,$0.74\pm0.06$,$30.65\pm3.53$,$0.65\pm0.03$,$0.75\pm0.01$,$8.98\pm0.69$


In [21]:
# Separando por grupos
display(Markdown('# Partição de treino'))

display(Markdown('## Grupo 1 - GSGP, GP, DCGP, ITEA, SymTree, FEAT'))
display(pd.concat([
    alg for alg in RMSES_train if alg.name in ['GSGP(novo)', 'gplearn', 'dcgp']
], axis=1).T.style.apply(highlight_min))

display(Markdown('## Grupo 2 - KNN, Forest, Tree'))
display(pd.concat([
    alg for alg in RMSES_train if alg.name in ['knn', 'forest', 'tree']
], axis=1).T.style.apply(highlight_min))

display(Markdown('## Grupo 3 - ElNet, Lasso, Lars, Ridge'))
display(pd.concat([
    alg for alg in RMSES_train if alg.name in ['elnet', 'lasso', 'lassolars', 'ridge']
], axis=1).T.style.apply(highlight_min))

# Partição de treino

## Grupo 1 - GSGP, GP, DCGP, ITEA, SymTree, FEAT

dataset,Geographical,airfoil,concrete,energyCooling,energyHeating,tecator,towerData,wineRed,wineWhite,yacht
GSGP(novo),$42.03\pm0.96$,$10.6\pm4.62$,$7.24\pm0.74$,$2.07\pm0.43$,$1.5\pm0.45$,$1.04\pm0.46$,$63.84\pm4.7$,$0.62\pm0.01$,$0.73\pm0.02$,$8.81\pm1.11$
gplearn,$49.66\pm2.58$,$10.28\pm7.26$,$13.64\pm1.89$,$4.44\pm0.91$,$4.67\pm0.92$,$3.0\pm0.63$,$68.34\pm5.87$,$0.79\pm0.06$,$0.87\pm0.05$,$6.38\pm2.45$
dcgp,$55.6\pm3.41$,$7.56\pm1.33$,$19.45\pm2.42$,$12.89\pm2.35$,$13.74\pm2.02$,$4.1\pm0.35$,$106.34\pm24.03$,$1.14\pm0.97$,$1.14\pm0.84$,$17.09\pm1.76$


## Grupo 2 - KNN, Forest, Tree

dataset,Geographical,airfoil,concrete,energyCooling,energyHeating,tecator,towerData,wineRed,wineWhite,yacht
forest,$11.26\pm0.76$,$1.8\pm0.03$,$2.97\pm0.07$,$1.22\pm0.03$,$0.4\pm0.01$,$0.63\pm0.02$,$12.97\pm0.42$,$0.34\pm0.0$,$0.53\pm0.0$,$0.4\pm0.02$
knn,$31.51\pm0.52$,$5.9\pm0.08$,$8.27\pm0.09$,$1.97\pm0.04$,$2.37\pm0.06$,$1.39\pm0.04$,$15.11\pm0.14$,$0.68\pm0.01$,$0.71\pm0.01$,$8.75\pm0.68$
tree,$19.73\pm3.5$,$2.04\pm0.08$,$2.9\pm0.16$,$1.68\pm0.07$,$0.34\pm0.03$,$0.66\pm0.21$,$13.55\pm0.52$,$0.6\pm0.01$,$0.7\pm0.01$,$0.33\pm0.41$


## Grupo 3 - ElNet, Lasso, Lars, Ridge

dataset,Geographical,airfoil,concrete,energyCooling,energyHeating,tecator,towerData,wineRed,wineWhite,yacht
elnet,$43.7\pm0.18$,$4.82\pm0.04$,$10.47\pm0.12$,$3.22\pm0.02$,$2.96\pm0.05$,$0.8\pm0.03$,$30.66\pm1.03$,$0.65\pm0.01$,$0.75\pm0.0$,$8.94\pm0.15$
lasso,$43.85\pm0.27$,$4.82\pm0.04$,$10.47\pm0.12$,$3.22\pm0.02$,$2.96\pm0.05$,$0.82\pm0.06$,$30.87\pm1.14$,$0.65\pm0.01$,$0.75\pm0.0$,$8.94\pm0.15$
lassolars,$44.42\pm0.11$,$4.82\pm0.04$,$10.47\pm0.12$,$3.22\pm0.02$,$2.96\pm0.05$,$0.8\pm0.06$,$30.65\pm1.23$,$0.65\pm0.01$,$0.75\pm0.0$,$8.95\pm0.16$
ridge,$43.72\pm0.26$,$4.82\pm0.04$,$10.47\pm0.12$,$3.22\pm0.02$,$2.96\pm0.05$,$0.8\pm0.04$,$30.31\pm1.05$,$0.65\pm0.01$,$0.75\pm0.0$,$8.95\pm0.16$


In [22]:
# Separando por grupos

display(Markdown('# Partição de teste'))

display(Markdown('## Grupo 1 - GSGP, GP, DCGP, ITEA, SymTree, FEAT'))
display(pd.concat([
    alg for alg in RMSES_test if alg.name in ['GSGP(novo)', 'gplearn', 'dcgp']
], axis=1).T.style.apply(highlight_min))

display(Markdown('## Grupo 2 - KNN, Forest, Tree'))
display(pd.concat([
    alg for alg in RMSES_test if alg.name in ['knn', 'forest', 'tree']
], axis=1).T.style.apply(highlight_min))

display(Markdown('## Grupo 3 - ElNet, Lasso, Lars, Ridge'))
display(pd.concat([
    alg for alg in RMSES_test if alg.name in ['elnet', 'lasso', 'lassolars', 'ridge']
], axis=1).T.style.apply(highlight_min))

# Partição de teste

## Grupo 1 - GSGP, GP, DCGP, ITEA, SymTree, FEAT

dataset,Geographical,airfoil,concrete,energyCooling,energyHeating,tecator,towerData,wineRed,wineWhite,yacht
GSGP(novo),$44.35\pm1.51$,$10.6\pm4.35$,$7.86\pm1.1$,$2.13\pm0.42$,$1.57\pm0.46$,$1.41\pm0.56$,$64.0\pm5.15$,$0.64\pm0.03$,$0.74\pm0.02$,$9.03\pm1.03$
gplearn,$49.72\pm2.94$,$10.31\pm7.21$,$13.65\pm2.09$,$4.46\pm0.9$,$4.67\pm0.97$,$3.05\pm0.7$,$68.61\pm6.59$,$0.79\pm0.07$,$0.88\pm0.05$,$6.68\pm2.94$
dcgp,$63.75\pm46.9$,$7.55\pm1.38$,$19.5\pm2.27$,$12.84\pm2.25$,$13.67\pm1.88$,$4.1\pm0.43$,$106.93\pm24.39$,$1.14\pm0.97$,$1.14\pm0.83$,$17.01\pm2.2$


## Grupo 2 - KNN, Forest, Tree

dataset,Geographical,airfoil,concrete,energyCooling,energyHeating,tecator,towerData,wineRed,wineWhite,yacht
forest,$25.78\pm3.65$,$2.37\pm0.07$,$5.25\pm0.32$,$1.79\pm0.12$,$0.57\pm0.1$,$1.25\pm0.12$,$17.69\pm1.27$,$0.58\pm0.04$,$0.66\pm0.01$,$1.07\pm0.31$
knn,$31.17\pm0.87$,$5.86\pm0.26$,$7.85\pm0.73$,$1.92\pm0.13$,$2.32\pm0.42$,$1.29\pm0.09$,$14.81\pm0.85$,$0.66\pm0.05$,$0.69\pm0.02$,$7.56\pm1.31$
tree,$31.92\pm7.42$,$2.93\pm0.1$,$6.5\pm0.5$,$1.85\pm0.14$,$0.57\pm0.07$,$1.68\pm0.15$,$20.09\pm1.02$,$0.69\pm0.06$,$0.75\pm0.02$,$1.44\pm0.26$


## Grupo 3 - ElNet, Lasso, Lars, Ridge

dataset,Geographical,airfoil,concrete,energyCooling,energyHeating,tecator,towerData,wineRed,wineWhite,yacht
elnet,$43.21\pm1.16$,$4.82\pm0.17$,$10.44\pm0.49$,$3.21\pm0.11$,$2.94\pm0.19$,$0.75\pm0.08$,$31.17\pm3.5$,$0.65\pm0.03$,$0.75\pm0.01$,$8.97\pm0.71$
lasso,$43.28\pm1.02$,$4.82\pm0.17$,$10.44\pm0.49$,$3.21\pm0.11$,$2.94\pm0.19$,$0.76\pm0.09$,$30.92\pm3.67$,$0.65\pm0.03$,$0.75\pm0.01$,$8.94\pm0.7$
lassolars,$43.92\pm1.01$,$4.82\pm0.17$,$10.44\pm0.49$,$3.21\pm0.11$,$2.94\pm0.19$,$0.74\pm0.08$,$30.56\pm3.46$,$0.65\pm0.03$,$0.75\pm0.01$,$8.93\pm0.71$
ridge,$42.95\pm1.26$,$4.82\pm0.17$,$10.44\pm0.49$,$3.21\pm0.11$,$2.94\pm0.19$,$0.74\pm0.06$,$30.65\pm3.53$,$0.65\pm0.03$,$0.75\pm0.01$,$8.98\pm0.69$
